In [94]:
# Super global constant variables
BOARD_ROW = 6
BOARD_COL = 7

In [95]:
import numpy as np
import random

class Connect4Env:
    def __init__(self):
        self.rows = BOARD_ROW
        self.cols = BOARD_COL
        self.board = np.zeros((self.rows, self.cols), dtype = np.int8)
        self.current_player = 1  # Player 1 = 1, Player 2 = 2
        self.reward = {'win': 10, 'draw': 0, 'lose': -10}

    def reset(self):
        self.__init__()
        return self.board.flatten()

    def play(self, action):
        """Thực hiện một hành động"""
        if action not in self.valid_moves():
            return self.board.flatten(), -10, True, {}  # Kết thúc nếu chọn sai

        for row in reversed(range(self.rows)):  # Tìm hàng trống thấp nhất
            if self.board[row, action] == 0:
                self.board[row, action] = self.current_player
                reward, done = self.isWinningMove()
                self.current_player = 3 - self.current_player  # Đổi lượt
                return self.board.flatten(), reward, done, {}

        return self.board.flatten(), -10, True, {}  # Kết thúc nếu cột đầy
    
    """Check whether there are 4 aligning discs
        Start with row (r), column (c)
        Moving in row/horizontal direction (dr), column/vertical direction (dc)
        player: The disc is of the same color/player"""
    def isWinningMove(self):
        def check_direction(r, c, dr, dc, player):
            count = 0
            for _ in range(4):
                if 0 <= r < self.rows and 0 <= c < self.cols and self.board[r, c] == player:
                    count += 1
                    r += dr
                    c += dc
                else:
                    break
            return count == 4

        for r in reversed(range(self.rows)):
            for c in range(self.cols):
                if self.board[r, c] != 0:
                    player = self.board[r, c]
                    if (check_direction(r, c, 1, 0, player) or  # Vertical
                            check_direction(r, c, 0, 1, player) or  # Horizontal
                            check_direction(r, c, 1, 1, player) or  # Diagonal /
                            check_direction(r, c, 1, -1, player)):  # Diagonal \
                        return (self.reward['win'], True)
                    
        # Check draw game
        if np.all(self.board != 0):
            return (self.reward['draw'], True)
        
        return (0, False)

    def valid_moves(self):
        """Trả về danh sách cột có thể đi"""
        return [c for c in range(self.cols) if self.board[0, c] == 0]
    
    def printBoard(self):
        for row in self.board:
            print(" ".join(["⚫" if x == 0 else "🚗" if x == 1 else "🚕" for x in row]))
        print(" 0  1   2  3   4  5   6")

board = Connect4Env()
state = board.reset()
print(state)
print(board.board)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0]
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]


In [96]:
import torch
import torch.nn as nn
import torch.optim as optim

class DQN(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(input_dim, 128)
        self.layer2 = nn.Linear(128, 128)
        self.layer3 = nn.Linear(128, output_dim)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        return self.layer3(x)  # Không dùng softmax vì DQN tối ưu giá trị Q


In [97]:
""" Use Epsilon-greedy to find out the best way to make a move

    @Params:
    @Return: best column to play"""
def select_action(model, state, epsilon):
    valid_moves = env.valid_moves()  # Lấy danh sách các cột hợp lệ

    if random.random() < epsilon:
        return np.random.choice(valid_moves)  # Chọn ngẫu nhiên trong các cột hợp lệ
    else:
        with torch.no_grad():
            q_values = model(torch.tensor(state, dtype=torch.float32).unsqueeze(0))
            q_values = q_values.numpy().flatten()  # Chuyển thành mảng numpy

        # Chỉ chọn hành động có Q-value cao nhất trong các cột hợp lệ
        best_action = max(valid_moves, key=lambda a: q_values[a])
        return best_action


In [98]:
""" Let 2 models play against each other
    @Return: reward after play optimally"""
def play_game(player1, player2, env, epsilon):
    state = env.reset()
    done = False
    turn = 1  # 1 là Player 1, 2 là Player 2

    while not done:
        model = player1 if turn == 1 else player2
        action = select_action(model, state, epsilon)
        
        next_state, reward, done, _ = env.step(action)

        if done:
            return reward  # Trả về kết quả trận đấu

        state = next_state
        turn = 3 - turn  # Đổi lượt chơi


In [99]:
from collections import deque

"""@Params: """
def train_model(model, optimizer, memory, batch_size):
    if len(memory) < batch_size:
        return

    batch = random.sample(memory, batch_size)
    
    for state, action, reward, next_state, done in batch:
        q_values = model(torch.tensor(state, dtype=torch.float32))
        q_value = q_values[action]

        with torch.no_grad():
            next_q_values = model(torch.tensor(next_state, dtype=torch.float32))
            target_q_value = reward if done else reward + 0.99 * torch.max(next_q_values)

        loss = (q_value - target_q_value) ** 2

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [100]:
env = Connect4Env()

input_dim = BOARD_ROW * BOARD_COL  # Size of board
output_dim = BOARD_COL  # Ouput action (column 0 -> n)

player1 = DQN(input_dim, output_dim)
player2 = DQN(input_dim, output_dim)

optimizer1 = optim.Adam(player1.parameters(), lr=0.001)
optimizer2 = optim.Adam(player2.parameters(), lr=0.001)

memory = deque(maxlen=10000)
batch_size = 64
num_episodes = 1
epsilon = 0.1

for episode in range(num_episodes):
    state = env.reset()
    done = False
    turn = 0  # 0 là Player 1, 1 là Player 2

    while not done:
        model = player1 if turn == 0 else player2
        optimizer = optimizer1 if turn == 0 else optimizer2

        action = select_action(model, state, epsilon)
        next_state, reward, done, _ = env.play(action)

        memory.append((state, action, reward, next_state, done))

        # Huấn luyện mô hình
        train_model(model, optimizer, memory, batch_size)

        state = next_state
        turn = 1 - turn  # Đổi lượt chơi

    if episode % 10 == 0:
        print(f"Episode {episode}: Reward {reward}")


print("Training complete!")


Episode 0: Reward 10
Training complete!


In [101]:
def evaluate_model(model, env, num_games):
    total_rewards = 0
    for _ in range(num_games):
        state = env.reset()
        done = False
        move_count = 0

        while not done:
            action = select_action(model, state, epsilon)  # Luôn chọn hành động tốt nhất
            state, reward, done, _ = env.play(action)
            env.printBoard()
            # print(f"🧐 Chọn cột: {action}")
            move_count +=1
            if move_count > 42:
                print("qua 42 buoc")
                print("Board state:\n", env.board)
                break
        
        total_rewards += reward  # Cộng tổng điểm thưởng
        
    avg_reward = total_rewards / num_games
    print(f"📊 Model's Average Reward over {num_games} games: {avg_reward}")
    return avg_reward

# Đánh giá model sau khi train
evaluate_model(player1, env, num_games=1)


⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ ⚫
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ ⚫
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ 🚗
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ 🚕
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ 🚗
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ 🚕
⚫ ⚫ 🚗 ⚫ ⚫ 🚗 🚗
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ 🚕
⚫ ⚫ 🚗 ⚫ ⚫ 🚗 🚗
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ 🚕
⚫ ⚫ 🚗 ⚫ ⚫ 🚗 🚗
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ 🚕
⚫ ⚫ 🚗 ⚫ ⚫ 🚗 🚗
 0  1   2  3   4  5   6
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ 🚕
⚫ ⚫ 🚗 ⚫ ⚫ 🚗 🚗
 0  1   2  3   4  5   6
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫


10.0

In [102]:
torch.save(player1.state_dict(), "dqn_player1.pth")
torch.save(player2.state_dict(), "dqn_player2.pth")
print("✅ Model saved successfully!")


✅ Model saved successfully!


In [103]:
player1.load_state_dict(torch.load("dqn_player1.pth"))
player2.load_state_dict(torch.load("dqn_player2.pth"))
print("✅ Model loaded successfully!")

✅ Model loaded successfully!


In [104]:
import time

start_time = time.time()
evaluate_model(player1, env, num_games=100)
end_time = time.time()

print(f"⏳ Time taken: {end_time - start_time:.2f} seconds")


⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ ⚫
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ ⚫
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ 🚗
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ 🚕
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ 🚗
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ 🚕
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ 🚗
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ 🚕
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ 🚗
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ 🚗
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ 🚕
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ 🚗
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ 🚗
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ 🚕
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ 🚗
 0  1   2  3   4  5   6
⚫ ⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ 🚗
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ 🚗
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ 🚕
⚫ ⚫ 🚗 ⚫ ⚫ ⚫ 🚗
 0  1   2  3   4  5   6
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ 🚕 ⚫ ⚫ ⚫ ⚫


In [105]:
import time

start = time.time()
# action = select_action(model, state, epsilon=0.0)
end = time.time()

print(f"⏳ Thời gian chọn action: {end - start:.4f} giây")


⏳ Thời gian chọn action: 0.0000 giây
